### Setup

In [1]:
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model.model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_fit_scenarios, do_create_multiple_reports
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = 'co-covid-models'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "co-covid-models-credentials.json"

/Users/emwu9912/PycharmProjects/covid-models


In [2]:
# set up the output directory for this Jupyter notebook
outdir = setup("20221118_scenario_projections.ipynb")

2022/11/15 22:04:44 INFO|============================================================


### Fit an initial scenario through February 2022

In [3]:
# designate the arguments for how the model will behave
model_args = {
    'params_defs': 'covid_model/analysis/20221109_nov_gov_briefing/20221109_scenario_params.json',
    'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/analysis/20221109_nov_gov_briefing/20221109_vacc_proj_params.json',
    'start_date': '2020-01-24',
    'end_date': '2024-01-01',
}

# this is how the fit will behave
# place the outdir argument here to tell the model fit where to go
fit_args = {'outdir': outdir,
            'fit_end_date': '2022-02-28'
}

# because all the scenarios are the same 
model = do_single_fit(**fit_args, **model_args)

2022/11/15 22:04:47 INFO|-------------------------|{} Retrieving vaccinations data
2022/11/15 22:04:49 INFO|-------------------------|{} Constructing vaccination projections
2022/11/15 22:04:51 INFO|-------------------------|{} Retrieving hospitalizations data
2022/11/15 22:04:54 INFO|----------------------|{} Prepping Model
2022/11/15 22:04:54 INFO|-----------------------|{} Prepping Model
2022/11/15 22:06:18 INFO|----------------------|{} Model prepped for fitting in 83.831551708 seconds.
2022/11/15 22:06:18 INFO|----------------------|{} Will fit 25 times
2022/11/15 22:07:19 INFO|----------------------|{'fit_batch': '0'}: Transmission control fit 1/25 completed in 61.46484262499999 seconds: {0: {'co': 0.8306366824121102}, 14: {'co': 0.698174348300508}, 28: {'co': 0.5695813526553709}, 42: {'co': 0.00278511917697288}, 56: {'co': 0.6814698187511562}, 70: {'co': 0.8912768746070765}}
2022/11/15 22:07:19 INFO|-----------------------|{'fit_batch': '0'}: Running forward sim
2022/11/15 22:07

2022/11/15 22:12:00 INFO|----------------------|{'fit_batch': '18'}: Transmission control fit 19/25 completed in 16.067380625 seconds: {504: {'co': 0.7926162886343049}, 518: {'co': 0.786203482523872}, 532: {'co': 0.7659880690937108}, 546: {'co': 0.7174287612461416}, 560: {'co': 0.7504800997336966}, 574: {'co': 0.7456518039217496}}
2022/11/15 22:12:00 INFO|-----------------------|{'fit_batch': '18'}: Running forward sim
2022/11/15 22:12:17 INFO|----------------------|{'fit_batch': '19'}: Transmission control fit 20/25 completed in 14.594883707999998 seconds: {532: {'co': 0.76534316964812}, 546: {'co': 0.7192434844758303}, 560: {'co': 0.7471077082985399}, 574: {'co': 0.7557418505542847}, 588: {'co': 0.792800462859069}, 602: {'co': 0.7737099015434532}}
2022/11/15 22:12:17 INFO|-----------------------|{'fit_batch': '19'}: Running forward sim
2022/11/15 22:12:31 INFO|----------------------|{'fit_batch': '20'}: Transmission control fit 21/25 completed in 12.409925791999967 seconds: {560: {'c

### Create and run scenarios from Feb 2022 to present

In [9]:
multiprocess = 3

scenario_params = json.load(open('covid_model/analysis/20221109_nov_gov_briefing/20221109_scenario_params.json'))

model_args = {
    'base_spec_id': 4386, # use the spec id that was output from the model fit
}
model_fit_args = {
    'outdir': outdir,
    'fit_start_date': '2022-03-01', # set the start date for the earliest point at which the scenarios start to differ from one another
    'pre_solve_model': True # force the model to establish initial conditions so the fit can start on the fit start date
}

# define vaccine effectiveness for < 5 (this is a multiplier for the baseline vaccine effectiveness for 0-19)
vacc_eff_lt5 = 0.5

# Create different scenarios to model
scenario_model_args = []
for (weak_escape, strong_escape) in [(0.6, 0.095), (0.55, 0.045), (0.65, 0.145)]:
    emv_seed = [{"param": "emv_seed", "attrs": None, "vals": {"2020-01-01": 0, "2022-09-15": 10,
                                                                       "2022-10-30": 0}}]
    hrf = {"2020-01-01": 1, "2022-03-01": (0.66 + 0.34*0.8),
           "2022-03-15": (0.34 + 0.66*0.8), "2022-03-30": 0.8}
    weak_param = [{"param": "immune_escape",
                   "from_attrs": {"immun": "weak",
                                  "variant": ["none", "wildtype", "alpha", "delta",
                                              "omicron", "ba2", "ba2121", "ba45"]},
                   "to_attrs": {"variant": ["emv"]},
                   "vals": {"2020-01-01": weak_escape},
                   "desc": "Scrabble variants immune escape value, weak immunity"}]
    strong_param = [{"param": "immune_escape",
                   "from_attrs": {"immun": "strong",
                                  "variant": ["none", "wildtype", "alpha", "delta",
                                              "omicron", "ba2", "ba2121", "ba45"]},
                   "to_attrs": {"variant": ["emv"]},
                   "vals": {"2020-01-01": strong_escape},
                   "desc": "Scrabble variants immune escape value, strong immunity"}]
    lt5_vacc_adjust = [{"param": "immunity",
                        "attrs": {'age': '0-19', 'vacc': 'shot1'},
                        "mults": {"2020-01-01": 1,
                                  "2022-06-24": 0.99 + 0.01*vacc_eff_lt5,
                                  "2022-06-30": 0.98 + 0.02*vacc_eff_lt5,
                                  "2022-07-08": 0.97 + 0.03*vacc_eff_lt5,
                                  "2022-07-19": 0.96 + 0.04*vacc_eff_lt5,
                                  "2022-07-29": 0.95 + 0.05*vacc_eff_lt5,
                                  "2022-08-11": 0.94 + 0.06*vacc_eff_lt5,
                                  "2022-08-30": 0.93 + 0.07*vacc_eff_lt5,
                                  "2022-09-26": 0.92 + 0.08*vacc_eff_lt5,
                                  "2022-10-26": 0.91 + 0.09*vacc_eff_lt5,},
                        "desc": "weighted average using share of 0-19 getting shot1 who are under 5"}]
    scenario_model_args.append({'params_defs': scenario_params + weak_param + strong_param + emv_seed + lt5_vacc_adjust,
                                'hosp_reporting_frac': hrf,
                                'tags': {'emv_escape_weak': weak_escape,
                                        'emv_escape_strong': strong_escape}})
            

In [10]:
# check how many scenarios there are
len(scenario_model_args)

3

In [11]:
# run the scenarios
models = do_fit_scenarios(base_model_args=model_args, scenario_args_list=scenario_model_args, fit_args=model_fit_args, multiprocess=multiprocess)

2022/11/15 22:33:41 INFO|============================================================
2022/11/15 22:33:41 INFO|============================================================
2022/11/15 22:33:41 INFO|============================================================
2022/11/15 22:33:45 INFO|-----------|{'emv_escape_weak': 0.6, 'emv_escape_strong': 0.095} Retrieving vaccinations data
2022/11/15 22:33:45 INFO|-----------|{'emv_escape_weak': 0.55, 'emv_escape_strong': 0.045} Retrieving vaccinations data
2022/11/15 22:33:45 INFO|-----------|{'emv_escape_weak': 0.65, 'emv_escape_strong': 0.145} Retrieving vaccinations data
2022/11/15 22:33:47 INFO|-----------|{'emv_escape_weak': 0.65, 'emv_escape_strong': 0.145} Constructing vaccination projections
2022/11/15 22:33:47 INFO|-----------|{'emv_escape_weak': 0.55, 'emv_escape_strong': 0.045} Constructing vaccination projections
2022/11/15 22:33:47 INFO|-----------|{'emv_escape_weak': 0.6, 'emv_escape_strong': 0.095} Constructing vaccination projections


2022/11/15 22:36:28 INFO|--------|{'emv_escape_weak': 0.6, 'emv_escape_strong': 0.095, 'fit_batch': '3'}: Transmission control fit 4/7 completed in 12.635247582999995 seconds: {851: {'co': 0.8266170566332579}, 865: {'co': 0.8525056500390544}, 879: {'co': 0.8387776063733919}, 893: {'co': 0.8329728220677577}, 907: {'co': 0.8382177230843638}, 921: {'co': 0.8311820089173593}}
2022/11/15 22:36:28 INFO|---------|{'emv_escape_weak': 0.6, 'emv_escape_strong': 0.095, 'fit_batch': '3'}: Running forward sim
2022/11/15 22:36:28 INFO|--------|{'emv_escape_weak': 0.65, 'emv_escape_strong': 0.145, 'fit_batch': '3'}: Transmission control fit 4/7 completed in 12.866401208000013 seconds: {851: {'co': 0.8266170566332579}, 865: {'co': 0.8525056500390544}, 879: {'co': 0.8387776063733919}, 893: {'co': 0.8329728220677577}, 907: {'co': 0.8382177230843638}, 921: {'co': 0.8311820089173593}}
2022/11/15 22:36:28 INFO|---------|{'emv_escape_weak': 0.65, 'emv_escape_strong': 0.145, 'fit_batch': '3'}: Running forwar

2022/11/15 22:37:15 INFO|--------|{'emv_escape_weak': 0.65, 'emv_escape_strong': 0.145, 'fit_batch': '6', 'run_type': 'fit'}: fitted TC: {0: {'co': 0.8306366815589404}, 14: {'co': 0.6981743540911125}, 28: {'co': 0.5650699126380538}, 42: {'co': 1.2613785476244924e-15}, 56: {'co': 0.6966530538855653}, 70: {'co': 0.8332726485652999}, 84: {'co': 0.8296248772646011}, 98: {'co': 0.8316924798351031}, 112: {'co': 0.8774896960445533}, 126: {'co': 0.8243044228361942}, 140: {'co': 0.7604527625947122}, 154: {'co': 0.671288017772332}, 168: {'co': 0.843055135293664}, 182: {'co': 0.7826363484044665}, 196: {'co': 0.8191511060028511}, 210: {'co': 0.7726071129497626}, 224: {'co': 0.7713215715743026}, 238: {'co': 0.718543964340705}, 252: {'co': 0.722097146662614}, 266: {'co': 0.6850776838461609}, 280: {'co': 0.7198394872192173}, 294: {'co': 0.7589434396271403}, 308: {'co': 0.8220715495341357}, 322: {'co': 0.8118499589368375}, 336: {'co': 0.7823102311026947}, 350: {'co': 0.7950241455590741}, 364: {'co': 0

### Run the report for each fit model

In [12]:
# here you can also specify which variants you want to calculate immunity for
do_create_multiple_reports(models, multiprocess=multiprocess, outdir=outdir, prep_model=False, solve_model=True, immun_variants=['ba45', 'emv'], from_date='2022-01-01')

2022/11/15 22:41:46 INFO|============================================================
2022/11/15 22:41:46 INFO|--------|Solving model
2022/11/15 22:41:47 INFO|============================================================
2022/11/15 22:41:47 INFO|--------|Solving model
2022/11/15 22:41:48 INFO|============================================================
2022/11/15 22:41:48 INFO|--------|Solving model
2022/11/15 22:41:51 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/11/15 22:41:51 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/11/15 22:41:52 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [13]:
logging.info('Projecting')
for model in models:
    logging.info('')
    #model.prep()  # don't think we need to prep anymore.
    model.solve_seir()

    model.solution_sum_df(['seir', 'variant', 'immun']).unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_seir_variant_immun_total_all_at_once_projection.csv')
    model.solution_sum_df().unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_full_projection.csv')

    logging.info(f'{str(model.tags)}: Running forward sim')
    fig = plt.figure(figsize=(10, 10), dpi=300)
    ax = fig.add_subplot(211)
    hosps_df = model.modeled_vs_observed_hosps().reset_index('region').drop(columns='region')
    hosps_df.plot(ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2024-01-01', "%Y-%m-%d").date())
    ax = fig.add_subplot(212)
    plot_transmission_control(model, ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2024-01-01', "%Y-%m-%d").date())
    plt.savefig(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast.png')
    plt.close()
    hosps_df.to_csv(get_filepath_prefix(outdir, tags=model.tags) + '_model_forecast.csv')
    json.dump(model.tc, open(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast_tc.json', 'w'))

logging.info('Running reports')

2022/11/15 22:46:59 INFO|Projecting
2022/11/15 22:46:59 INFO|
2022/11/15 22:47:24 INFO|{'emv_escape_weak': 0.6, 'emv_escape_strong': 0.095, 'fit_batch': '6', 'run_type': 'fit'}: Running forward sim
2022/11/15 22:47:26 INFO|
2022/11/15 22:47:50 INFO|{'emv_escape_weak': 0.55, 'emv_escape_strong': 0.045, 'fit_batch': '6', 'run_type': 'fit'}: Running forward sim
2022/11/15 22:47:51 INFO|
2022/11/15 22:48:13 INFO|{'emv_escape_weak': 0.65, 'emv_escape_strong': 0.145, 'fit_batch': '6', 'run_type': 'fit'}: Running forward sim
2022/11/15 22:48:15 INFO|Running reports


In [ ]:
model.solution_ydf.to_csv('covid_model/output/20221109_scenario_projections.ipynb/state_model_solution_ydf.csv', index=False)